# Run with a spinup and GCM data

The initial state of glaciers play a large role for the model output. In this example we illustrate how to “spinup” a glacier (e.g.: make them grow) before running over the period of interest. For this example we use climate data from the CESM Last Millennium Ensemble.

In [ ]:
# Libs
import matplotlib.pyplot as plt

# Locals
import oggm.cfg as cfg
from oggm import tasks, utils, workflow
from oggm.workflow import execute_entity_task
from oggm.utils import get_demo_file

In [ ]:
# Initialize OGGM and set up the default run parameters
cfg.initialize()

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('OGGM_spinup_run')
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR

# Use multiprocessing?
cfg.PARAMS['use_multiprocessing'] = False

Pre-processed directories are being used here.

In [ ]:
# How many grid points around the glacier?
# Make it large if you expect your glaciers to grow large
cfg.PARAMS['border'] = 80

# Go - initialize glacier directories
gdirs = workflow.init_glacier_regions(['RGI60-11.00897'], from_prepro_level=4)

Here the paths to the CESM-LME files are set. (The demo files that are being used in this example don't contain the whole last millennium, neither do they have the global coverage that they original files have. These demo files have been made for test purposes and to reduce the time it takes to run the example. If you use the demo files for a glacier outside the domain, you won't get an error. Instead the climate of the nearest point to the glacier that is available in the demo files will be used, which could be thousands of kilometers away.)

In [ ]:
# Additional climate file (CESM)
cfg.PATHS['cesm_temp_file'] = get_demo_file('cesm.TREFHT.160001-200512'
                                            '.selection.nc')
cfg.PATHS['cesm_precc_file'] = get_demo_file('cesm.PRECC.160001-200512'
                                             '.selection.nc')
cfg.PATHS['cesm_precl_file'] = get_demo_file('cesm.PRECL.160001-200512'
                                             '.selection.nc')
execute_entity_task(tasks.process_cesm_data, gdirs);

Here the CESM-LME data is being pre-processed. This process makes use of the delta method and uses scaled temperature anomalies by default (it is strongly recommended to use this default setting of scaling the temperature anomalies, unless you have very good reasons not to do so).

In [ ]:
execute_entity_task(tasks.process_cesm_data, gdirs)

In [ ]:
# Run the last 200 years with the default starting point (current glacier)
# and CESM data as input
execute_entity_task(tasks.run_from_climate_data, gdirs,
                    climate_filename='gcm_data',
                    ys=1801, ye=2000,
                    output_filesuffix='_no_spinup')

In [ ]:
# Run the spinup simulation - t* climate with a cold temperature bias
execute_entity_task(tasks.run_constant_climate, gdirs,
                    nyears=100, bias=0, temperature_bias=-0.5,
                    output_filesuffix='_spinup')
# Run a past climate run based on this spinup
execute_entity_task(tasks.run_from_climate_data, gdirs,
                    climate_filename='gcm_data',
                    ys=1801, ye=2000,
                    init_model_filesuffix='_spinup',
                    output_filesuffix='_with_spinup')

When starting from a spin-up, by default the last year of the spin-up is being used to initialize a glacier. With `init_model_yr`, you can select any other year from the spin-up as initial year.

In [ ]:
# Run a past climate run based on this spinup
execute_entity_task(tasks.run_from_climate_data, gdirs,
                    climate_filename='gcm_data',
                    ys=1801, ye=2000, init_model_yr=50,
                    init_model_filesuffix='_spinup',
                    output_filesuffix='_with_spinup_50yr')

In [ ]:
# Compile output
utils.compile_glacier_statistics(gdirs)
ds1 = utils.compile_run_output(gdirs, input_filesuffix='_no_spinup')
ds2 = utils.compile_run_output(gdirs, input_filesuffix='_with_spinup')
ds3 = utils.compile_run_output(gdirs, input_filesuffix='_with_spinup_50yr')

In [ ]:
# Plot
f, ax = plt.subplots(figsize=(9, 4))
(ds1.volume.sum(dim='rgi_id') * 1e-9).plot(ax=ax, label='No spinup')
(ds2.volume.sum(dim='rgi_id') * 1e-9).plot(ax=ax, label='With 100-yr spinup')
(ds3.volume.sum(dim='rgi_id') * 1e-9).plot(ax=ax, label='With 50-yr spinup')
ax.set_ylabel('Volume (km$^3$)')
ax.set_xlabel('')
ax.set_title('Hintereisferner volume under CESM-LME forcing')
plt.legend()
plt.tight_layout()

## What's next?

- look on the tutorial of how a typical "projection run" (there CMIP5) is done at [run_with_gcm.ipynb](run_with_gcm.ipynb)
- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)